In [15]:
import re
import pandas as pd
import os
!pip install pyth
from pyth.plugins.rtf15.reader import Rtf15Reader
from pyth.plugins.plaintext.writer import PlaintextWriter

def read_rtf(file_path):
    """Read RTF file and convert it to plain text."""
    with open(file_path, 'rb') as file:
        doc = Rtf15Reader.read(file)
        text = PlaintextWriter.write(doc).getvalue()
    return text

def parse_rtf_content(text):
    """Parse RTF content to extract variable information."""
    # Extract the file name
    file_name_match = re.search(r"File Name =\s+([^\n]+)", text)
    file_name = file_name_match.group(1).strip() if file_name_match else "Unknown"

    # Extract variables
    data = []
    # Regex to find each variable block
    variable_blocks = re.finditer(r"Pos\. = (\d+).*?Variable = (.*?)\s+Variable label = (.*?)(?=\n\nPos\. = |\Z)", text, re.DOTALL)
    for block in variable_blocks:
        position = block.group(1).strip()
        variable = block.group(2).strip()
        variable_label = block.group(3).strip()

        # Find data type and measurement level
        data_type_match = re.search(r"This variable is\s+(\w+), the SPSS measurement level is (\w+)", block.group(0))
        data_type = f"{data_type_match.group(1)}, {data_type_match.group(2)}" if data_type_match else "Unknown"

        # Find value labels
        value_labels = []
        value_label_matches = re.finditer(r"Value = (.*?)\s+Label = (.*?)\n", block.group(0))
        for vl_match in value_label_matches:
            value = vl_match.group(1).strip()
            label = vl_match.group(2).strip()
            value_labels.append(f"{value} = {label}")

        value_label_info = "; ".join(value_labels)

        data.append({
            "file_name": file_name,
            "position": position,
            "variable": variable,
            "variable_label": variable_label,
            "data_type": data_type,
            "value_label_info": value_label_info
        })

    return data

# Paths to the RTF files
#directory_9m = '/Users/faith/Desktop/Work/Boston Childrens Hospital/birth-cohort-db/UKMCS/data_dump/UKDA-4683-tab-9m/mrdoc/ukda_data_dictionaries'
#file_paths = [os.path.join(directory_9m, file) for file in os.listdir(directory_9m) if os.path.isfile(os.path.join(directory_9m, file))]
file_paths =['/Users/faith/Desktop/Work/Boston Childrens Hospital/birth-cohort-db/UKMCS/data_dump/UKDA-4683-tab-9m/mrdoc/ukda_data_dictionaries/mcs1_cm_derived_ukda_data_dictionary.rtf', '/Users/faith/Desktop/Work/Boston Childrens Hospital/birth-cohort-db/UKMCS/data_dump/UKDA-4683-tab-9m/mrdoc/ukda_data_dictionaries/mcs1_cm_derived_ukda_data_dictionary.rtf', '/Users/faith/Desktop/Work/Boston Childrens Hospital/birth-cohort-db/UKMCS/data_dump/UKDA-4683-tab-9m/mrdoc/ukda_data_dictionaries/mcs1_cm_derived_ukda_data_dictionary.rtf']
all_data = []

for file_path in file_paths:
    content = read_rtf(file_path)
    all_data.extend(parse_rtf_content(content))

# Create a DataFrame and save to CSV
df = pd.DataFrame(all_data)
csv_path = '/Users/faith/Desktop/Work/Boston Childrens Hospital/birth-cohort-db/UKMCS/code/extracted_data.csv'
df.to_csv(csv_path, index=False)

SyntaxError: invalid syntax (reader.py, line 653)